#  MODIS Water Cluster Training

Version: 0.1.0

Date modified: 05.01.2023

Modified by: Amanda Burke

In [1]:
import csv
import datetime
import glob
import joblib
import numpy as np
import os
import pandas as pd
from pathlib import Path   
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split 

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline


import optuna
from sklearn.ensemble import RandomForestClassifier as skRF
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, f1_score
from sklearn.metrics import classification_report, roc_curve, auc, matthews_corrcoef
from sklearn.model_selection import RandomizedSearchCV, KFold, StratifiedKFold
#from sklearn.inspection import permutation_importance


# #GDAL Stuff
# from osgeo import gdalconst
# from osgeo import gdal
# from pprint import pprint

# GPU-based frameworks

import cudf
import cupy as cp
from cuml.ensemble import RandomForestClassifier as cuRFC

In [2]:
GPU = False
MODEL = 'rf'
TEST_RATIO = 0.2
RANDOM_STATE = 42
LABEL_NAME = 'water'
DATA_TYPE = np.int16
FRAC_LAND=0.5
num_datapoints = 10000000

In [3]:
# #############################
# # VERSION 4.2.1 (targeted 500k points)
# TILE_IN = 'Golden'#v4.2.1
# DATA_VERSION='v4.2.1'
# offsets_indexes = ['x_offset', 'y_offset', 'year', 'julian_day','tileID']
# #############################

##############################
#VERSION 2.0.1 (5 million points)
TILE_IN = 'GLOBAL'#v2.0.1
DATA_VERSION='v2.0.1'
offsets_indexes = ['x_offset', 'y_offset', 'year', 'julian_day']
##############################

# #############################
# #VERSION 0.0.0 (2billion data points)
# TILE_IN = 'cleaned'#v2.0.1
# DATA_VERSION='AGU'
# offsets_indexes = []#'x_offset', 'y_offset', 'year', 'julian_day']
# ##############################

training_data_basepath = f'/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/{DATA_VERSION}'
glob_string = os.path.join(training_data_basepath,'MOD*{}*.parquet.gzip'.format(TILE_IN))
data_paths = sorted([fv for fv in glob.glob(glob_string)])

#Only want the one with 4.2.0 because the other file doesnt work. 
print(data_paths)
data_path = data_paths[0]
print(data_path)

['/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/v2.0.1/MOD09_GLOBAL_5469777_2_0_1.parquet.gzip']
/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/v2.0.1/MOD09_GLOBAL_5469777_2_0_1.parquet.gzip


In [4]:
def load_cpu_data(fpath, colsToDrop, yCol='water', testSize=0.2, randomState=42, 
              dataType=np.float32, cpu=True, splitXY=False, trainTestSplit=False,
             applyLog=False, imbalance=False, frac=0.1, land=False, multi=False, 
              multisample=1000000):
    """
    Simple helper function for loading data to be used by models
    :param fpath: Path to the data to be ingested.
    :param dataType: Data type to convert ingested data to.
    :param colsToDrop: Columns which are not necessary, from which to drop.
    :param testSize: Ration to
    """
    if multi:
        all_dfs = [pd.read_csv(path_) for path_ in fpath]
        df = pd.concat(all_dfs).sample(n=multisample, random_state=randomState)
        print('DF length: {}'.format(len(df.index)))
    else:   
        df = pd.read_parquet(fpath) if '.parquet' in fpath else pd.read_csv(fpath)
    df = df[df['sur_refl_b01_1'] + df['sur_refl_b02_1'] != 0]
    df = df[df['sur_refl_b07_1'] + df['sur_refl_b02_1'] != 0]
    df = df[df['sur_refl_b06_1'] + df['sur_refl_b02_1'] != 0]

    df = df.drop(columns=colsToDrop)
    cleanedDF = df[~df.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0).astype(dataType)
    if applyLog:
        for col in cleanedDF.drop([yCol], axis=1).columns:
            print('Applying log1p func to {}'.format(col))
            cleanedDF[col] = np.log1p(cleanedDF[col])
        cleanedDF = cleanedDF[~cleanedDF.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0)
    df = None
    if imbalance:
        if land:
            print('Imbalancing data, sampling {} from water'.format(frac))
        else:
            print(f'Imbalancing data, sampling {frac} from land, {1-frac} from water')
        groupedDF = cleanedDF.groupby('water')
        dfs = [groupedDF.get_group(y) for y in groupedDF.groups]
        sampledDF = dfs[1].sample(frac=frac)if land else dfs[0].sample(frac=frac)
        concatDF = sampledDF.append(dfs[0]) if land else sampledDF.append(dfs[1])
        concatDF = concatDF.sample(frac=1)
        concatDF = concatDF.reset_index()
        cleanedDF = concatDF.drop(columns=['index'])
    if not splitXY:
        return cleanedDF
    X = cleanedDF.drop([yCol], axis=1).astype(dataType)
    y = cleanedDF[yCol].astype(dataType)
    if trainTestSplit:
        return train_test_split(X, y, test_size=TEST_RATIO)
    else:
        return X, y

In [5]:
colsToDrop = [
            #'sur_refl_b01_1',
            # 'sur_refl_b02_1',
             'sur_refl_b03_1',
             'sur_refl_b04_1','sur_refl_b05_1','sur_refl_b06_1',
            # 'sur_refl_b07_1',
             # 'ndvi',
             'ndwi1','ndwi2'
            ]

colsToDropTraining = colsToDrop.copy()
colsToDropTraining.extend(offsets_indexes)
v_names = ['sur_refl_b01_1','sur_refl_b02_1','sur_refl_b03_1',
           'sur_refl_b04_1','sur_refl_b05_1','sur_refl_b06_1',
           'sur_refl_b07_1','ndvi','ndwi1','ndwi2']

### Input data

In [6]:
colsToDrop

['sur_refl_b03_1',
 'sur_refl_b04_1',
 'sur_refl_b05_1',
 'sur_refl_b06_1',
 'ndwi1',
 'ndwi2']

In [30]:
%%time
X, X_test, y, y_test = load_gpu_data(fpath=data_path,
                                             colsToDrop=colsToDropTraining,
                                             dataType=cp.float32,
                                             cpu=False,
                                             splitXY=True,
                                             imbalance=False,
                                             trainTestSplit=True)
X = X.iloc[:num_datapoints,:] 
y = y.iloc[:num_datapoints] 

X_test = X_test.iloc[:num_datapoints,:] 
y_test = y_test.iloc[:num_datapoints] 

print(f'data shape: {X.shape}, {y.shape}')

data shape: (4375821, 4), (4375821,)
CPU times: user 3.9 s, sys: 1.05 s, total: 4.95 s
Wall time: 4.14 s


In [31]:
# #Getting the indices that are associated with land (0) and water (1)
# y_water_ind = np.where(y>0.5)[0]
# y_land_ind = np.where(y<0.5)[0]

# #Subset the X AND y data to later/ subset with the clusters and then combine for RFA
# X_water = X.iloc[y_water_ind,:]
# y_water = y.iloc[y_water_ind]

# X_land = X.iloc[y_land_ind,:]
# y_land = y.iloc[y_land_ind]

TypeError: no implementation found for 'numpy.where' on types that implement __array_function__: [<class 'cudf.core.series.Series'>]

In [9]:
_ = [print(column) for column in X.columns]

sur_refl_b01_1
sur_refl_b02_1
sur_refl_b07_1
ndvi


### Comparing targeted and clustered samples


In [ ]:
DATA_VERSION='v4.2.1'
training_data_basepath = f'/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/{DATA_VERSION}'

#VERSION 4.2.1
TILE_IN = 'Golden'#v4.2.1
offsets_indexes = ['x_offset', 'y_offset', 'year', 'julian_day','tileID']

glob_string = os.path.join(training_data_basepath,'MOD*{}*.parquet.gzip'.format(TILE_IN))
data_paths = sorted([fv for fv in glob.glob(glob_string)])

#Only want the one with 4.2.0 because the other file doesnt work. 
print(data_paths)
data_path = data_paths[0]
print(data_path)
colsToDropTraining = colsToDrop.copy()
colsToDropTraining.extend(offsets_indexes)


X_target, X_test_target, y_target, y_test_target = load_data(fpath=data_path,
                                colsToDrop=colsToDropTraining,
                                dataType=DATA_TYPE,
                                cpu=True,
                                splitXY=True,
                                trainTestSplit=True
                                )

X_target = X_target.iloc[:num_datapoints,:] 
y_target = y_target.iloc[:num_datapoints] 

X_test_target = X_test_target.iloc[:num_datapoints,:] 
y_test_target = y_test_target.iloc[:num_datapoints] 

print(f'\n\ntarget subset data shape: {X_target.shape}, {y_target.shape}')

#Getting the indices that are associated with land (0) and water (1)
#Subset the X AND y data to later subset with the clusters and then combine for RFA
X_water_target = X_target.iloc[np.where(y_target>0.5)[0],:]
X_land_target = X_target.iloc[np.where(y_target<0.5)[0],:]

### Clustering Data for Input to Random Forest

Based on the cluster analysis above on 5.03.23, 15 clusters appears to have the most data and exclude outliers so will use that number for selection 

In [ ]:
CLUSTER_NUM=15

common_params = {
    "n_init": "auto",
    # "random_state": 42,
    "init":"random"
}

In [ ]:
%%time
kme_land_random =  KMeans(n_clusters=CLUSTER_NUM, **common_params).fit(X_land)
kmeans_output_land_random = kme_land_random.predict(X_land)

In [ ]:
%%time
kme_water_random = KMeans(n_clusters=CLUSTER_NUM, **common_params).fit(X_water)
kmeans_output_water_random = kme_water_random.predict(X_water)

### Even Balanced Random pulled datapoints

In [ ]:
COUNT_EVEN_BALANCE_LAND = np.inf
COUNT_EVEN_BALANCE_WATER = np.inf
for cluster in np.unique(kmeans_output_water_random):
    land_num = len(np.where(kmeans_output_land_random == cluster)[0])
    water_num = len(np.where(kmeans_output_water_random == cluster)[0])
    if land_num < COUNT_EVEN_BALANCE_LAND: COUNT_EVEN_BALANCE_LAND = land_num
    if water_num < COUNT_EVEN_BALANCE_WATER: COUNT_EVEN_BALANCE_WATER = water_num
    
print(COUNT_EVEN_BALANCE_LAND, COUNT_EVEN_BALANCE_WATER)
if COUNT_EVEN_BALANCE_LAND < COUNT_EVEN_BALANCE_WATER:
    COUNT = COUNT_EVEN_BALANCE_LAND
else: 
    COUNT = COUNT_EVEN_BALANCE_WATER
print(COUNT,COUNT_EVEN_BALANCE_LAND,COUNT_EVEN_BALANCE_WATER)

In [ ]:
# np.random.seed(42)
random_ind_land = np.array([])
random_ind_water = []

for cluster in np.unique(kmeans_output_water_random):
    print(f'cluster {cluster}')
    cluster_ind_water = np.where(kmeans_output_water_random == cluster)[0]
    random_pts_water = np.random.choice(cluster_ind_water,COUNT,replace=False)
    max_X_random_water = np.nanmax(X_water['sur_refl_b01_1'].iloc[random_pts_water])
    if max_X_random_water < 10000:
        random_ind_water = np.append(random_ind_water, random_pts_water)
    else: print(f'Cluster {cluster} contains outliers')
    
    cluster_ind_land = np.where(kmeans_output_land_random == cluster)[0]
    random_pts_land = np.random.choice(cluster_ind_land,COUNT,replace=False)
    random_ind_land = np.append(random_ind_land, random_pts_land)
    
random_ind_water = random_ind_water.astype('int')
random_ind_land = random_ind_land.astype('int')

print(random_ind_water,random_ind_land)

In [ ]:
X_cluster_land_random = X_land.iloc[random_ind_land]
y_cluster_land_random = y_land.iloc[random_ind_land]
X_cluster_water_random = X_water.iloc[random_ind_water]
y_cluster_water_random = y_water.iloc[random_ind_water]

X_cluster_random = pd.concat([X_cluster_land_random,X_cluster_water_random])
y_cluster_random = pd.concat([y_cluster_land_random,y_cluster_water_random])

#Combine the data so that we can shuffle the indices and keep the data together that should be
All_data_random = pd.concat([X_cluster_random,y_cluster_random],axis=1).sample(frac=1)

X_cluster_rfa_random = All_data_random[X_cluster_random.columns]
y_cluster_rfa_random = All_data_random['water']

In [ ]:
print(len(X_cluster_land_random),len(X_cluster_water_random))

In [ ]:
print(All_data_random)
print(X_cluster_rfa_random)
print(y_cluster_rfa_random)

### Percentage Random pulled datapoints

In [ ]:
# List of the clusters: kmeans_output_land and kmeans_output_water
# Data: X_water, X_land, y_water, y_land

PERCENT_RANDOM_PULL = 0.15

In [ ]:
# np.random.seed(42)
random_ind_land = np.array([])
random_ind_water = []

for cluster in np.unique(kmeans_output_water_random):
    print(f'cluster {cluster}')
    cluster_ind_water = np.where(kmeans_output_water_random == cluster)[0]
    # cluster_ind_water = np.where(bgm_water == cluster)[0]
    COUNT_RANDOM_PULL_WATER = int(PERCENT_RANDOM_PULL*len(cluster_ind_water))
    random_pts_water = np.random.choice(cluster_ind_water,COUNT_RANDOM_PULL_WATER,replace=False)
    max_X_random_water = np.nanmax(X_water['sur_refl_b01_1'].iloc[random_pts_water])
    if max_X_random_water < 10000:
        random_ind_water = np.append(random_ind_water, random_pts_water)
    else: print(f'Cluster {cluster} contains outliers')
    
    cluster_ind_land = np.where(kmeans_output_land_random == cluster)[0]
    # cluster_ind_land = np.where(bgm_land == cluster)[0]
    COUNT_RANDOM_PULL_LAND = int(PERCENT_RANDOM_PULL*len(cluster_ind_land))
    random_pts_land = np.random.choice(cluster_ind_land,COUNT_RANDOM_PULL_LAND,replace=False)
    random_ind_land = np.append(random_ind_land, random_pts_land)
    print(f'Pulling {COUNT_RANDOM_PULL_WATER} Water pts and {COUNT_RANDOM_PULL_LAND} Land pts')
    print()
random_ind_water = random_ind_water.astype('int')
random_ind_land = random_ind_land.astype('int')

print(random_ind_water,random_ind_land)

#### Total random dataset used for training random forest

In [ ]:
X_cluster_land_random = X_land.iloc[random_ind_land]
y_cluster_land_random = y_land.iloc[random_ind_land]
X_cluster_water_random = X_water.iloc[random_ind_water]
y_cluster_water_random = y_water.iloc[random_ind_water]

X_cluster_random = pd.concat([X_cluster_land_random,X_cluster_water_random])
y_cluster_random = pd.concat([y_cluster_land_random,y_cluster_water_random])

#Combine the data so that we can shuffle the indices and keep the data together that should be
All_data_random = pd.concat([X_cluster_random,y_cluster_random],axis=1).sample(frac=1)

X_cluster_rfa_random = All_data_random[X_cluster_random.columns]
y_cluster_rfa_random = All_data_random['water']

In [ ]:
print(len(X_cluster_land_random),len(X_cluster_water_random))

## Random forest

In [ ]:
# FIGURE_OUTPUT_DIR = ''#/explore/nobackup/projects/ilab/scratch/cssprad1/MODIS_water/code/tmp'
# RASTER_OUTPUT_DIR = ''#/explore/nobackup/projects/ilab/scratch/cssprad1/MODIS_water/code/tmp'
# MODEL_OUTPUT_DIR = ''#/explore/nobackup/projects/ilab/scratch/cssprad1/MODIS_water/models/'

# qaMaskPath = '/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/qa'
# waterMaskPath = '/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs'

# test_data_basepath = '/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/test_data/'

# ##Add in the other stuff for tuning
# frac_water = int(100*(1.0-FRAC_LAND))
# if len(colsToDrop) >= 1:
#     save_file = f"{TILE}_{DAY}_{YEAR}_frac-water{frac_water}_no-vars_{'-'.join(colsToDrop)}"
# else: 
#      save_file = f"{TILE}_{DAY}_{YEAR}_frac-water{frac_water}_all-vars"

# print(save_file)

#### RF FUNCTIONS

In [14]:
def cpu_rf_objective(trial):
    list_trees = [75, 100, 125, 150, 175, 200, 250, 300, 400, 500]
    max_depth = [5, 10, 30, 50, 80, 90, 100, 110]
    min_samples_leaf = [1, 2, 3, 4, 5]
    min_samples_split = [2, 4, 8, 10]
    bootstrap = [True, False]
    max_features = ['auto', 'sqrt', 'log2']
    
    # if TREES_AND_DEPTH_ONLY:
    #     param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
    #                'criterion':'gini', 
    #                'max_depth':trial.suggest_categorical('max_depth', max_depth), 
    #                'min_samples_split':2, 
    #                'min_samples_leaf':1, 
    #                'min_weight_fraction_leaf':0.0, 
    #                'max_features':'auto', 
    #                'max_leaf_nodes':None, 
    #                'min_impurity_decrease':0.0, 
    #                'bootstrap':True, 
    #                'oob_score':False, 
    #                'n_jobs':-1, 
    #                # 'random_state':42, 
    #                'verbose':0, 
    #                'warm_start':True, 
    #                'class_weight':None, 
    #                'ccp_alpha':0.0, 
    #                'max_samples':None
    #                   }
    # else:
    param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
                       'max_depth':trial.suggest_categorical('max_depth', max_depth), 
                       'min_samples_split':trial.suggest_categorical('min_samples_split', min_samples_split), 
                       'min_samples_leaf':trial.suggest_categorical('min_samples_leaf', min_samples_leaf), 
                       'bootstrap': trial.suggest_categorical('bootstrap', bootstrap),
                       'criterion':'gini', 
                       #'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-8, 1.0, log=True), 
                       'max_features':trial.suggest_categorical('max_features', max_features), 
                       'max_leaf_nodes':None, 
                       'min_impurity_decrease':0.0, 
                       'oob_score':False, 
                       'n_jobs':-1, 
                       # 'random_state':42, 
                       'verbose':0, 
                       'warm_start':False, 
                       'class_weight':None, 
                       'ccp_alpha':0.0, 
                       'max_samples':None
                      }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    #######################
    # HERE IS WHERE TO CHANGE THE X,Y DATASET USED FOR TRAINING
    #######################
   
    cv_scores = np.empty(5)
    # for idx, (train_idx, val_idx) in enumerate(cv.split(X,y)):
    #     # X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    #     # y_train, y_val = y.iloc[train_idx],  y.iloc[val_idx]

    for idx, (train_idx, val_idx) in enumerate(cv.split(X_cluster_rfa_random,  y_cluster_rfa_random)):    
        X_train, X_val = X_cluster_rfa_random.iloc[train_idx], X_cluster_rfa_random.iloc[val_idx]
        y_train, y_val = y_cluster_rfa_random.iloc[train_idx],  y_cluster_rfa_random.iloc[val_idx]
        
    # for idx, (train_idx, val_idx) in enumerate(cv.split(X_cluster_rfa_center, y_cluster_rfa_center)):
    #     X_train, X_val = X_cluster_rfa_center.iloc[train_idx], X_cluster_rfa_center.iloc[val_idx]
    #     y_train, y_val = y_cluster_rfa_center.iloc[train_idx], y_cluster_rfa_center.iloc[val_idx]

        model = skRF(**param)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        cv_scores[idx] = f1_score(y_val, preds)
        if cv_scores[idx] == 0.0:
            print('Pruning because of 0.0 score.')
            return 0.0
        print('Fold {}: {}'.format(idx, cv_scores[idx]))
    return np.mean(cv_scores)

search_space={
    "n_estimators": [75, 100, 125, 150, 175, 200, 250, 300, 400, 500],
    "max_depth" : [5,10, 30, 50, 80, 90, 100, 110],
    "min_samples_leaf" : [1, 2, 3, 4, 5],
    "min_samples_split" : [2, 4, 8, 10],
    "bootstrap" : [True, False],
    "max_features" : ['auto', 'sqrt', 'log2'],
    
}
TREES_AND_DEPTH_ONLY = False
GRID_SEARCH = True

In [34]:
print(X)

         sur_refl_b01_1  sur_refl_b02_1  sur_refl_b07_1          ndvi
2503850          1932.0          3032.0          2755.0   2215.954834
5087060          1104.0          2616.0          2286.0   4064.516113
3343370            17.0            11.0             1.0  -2142.857178
2091537           398.0          2522.0           924.0   7273.972656
1744767           198.0           170.0            80.0   -760.869568
...                 ...             ...             ...           ...
1292987           556.0          2078.0          1177.0   5778.284180
4525991            13.0             1.0            43.0  -8571.428711
2615424          1170.0          3158.0          2104.0   4593.345703
1222149            34.0             2.0            20.0  -8888.888672
3752190             3.0           -15.0            13.0  15000.000000

[4375821 rows x 4 columns]


#### Training RF

In [35]:
%%time

optuna.logging.set_verbosity(optuna.logging.INFO)
if GRID_SEARCH:
    study = optuna.create_study(study_name='RF Tuning Grid Search', 
                                direction='maximize',
                                sampler=optuna.samplers.GridSampler(search_space))
    
else:
    study = optuna.create_study(study_name='RF Tuning',
                                direction='maximize')
#Objective is under the functions area

#####################################################################
#CHANGE HERE FOR DIFFERENT MODELING TYPE
#rf_objective or xgb_objective
#####################################################################

study.optimize(cpu_rf_objective, n_trials=25, timeout=30*600)

[I 2023-08-01 13:44:42,756] A new study created in memory with name: RF Tuning
[W 2023-08-01 13:44:42,759] Trial 0 failed with parameters: {'n_estimators': 100, 'max_depth': 30} because of the following error: TypeError('Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()\nTo explicitly construct a host matrix, consider using .to_numpy().').
Traceback (most recent call last):
  File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2599613/1274776373.py", line 27, in gpu_rf_objective
    for idx, (train_idx, val_idx) in enumerate(cv.split(X,y)):
  File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/sklearn/model_selection/_split.py", line 771, in split
    y = check_array(y, input_name="y", ensure_2d=False, dtype=None)
  F

TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
trials = study.best_trials
trial_score = max([trial.values[0]
                   for trial in trials])
best_trial_params = [trial.params for trial in trials if trial.values[0] == trial_score][0]
print(best_trial_params)
print(trial_score)
score_print = np.round(trial_score,4)
print(score_print)

In [ ]:
print(len(X_cluster_land_random),len(X_cluster_water_random),trial_score )

In [ ]:
hyperparameters = best_trial_params
hyperparameters['n_jobs'] = -1
print('Using these params:')
print(hyperparameters)
tuned_classifier = skRF(**hyperparameters)

In [ ]:
%%time 
tuned_classifier.fit(X_cluster_rfa_random , y_cluster_rfa_random)

In [ ]:
import pickle
# save the model to disk
filename = 'rfa_models/MODIS_RFA_v201_PercentCluster_sfcref127ndvi_10.pkl'
print(filename)
pickle.dump(tuned_classifier, open(filename, 'wb'))

In [ ]:
pickled_model = pickle.load(open('rfa_models/MODIS_RFA_v201_EBCluster_sfcref127ndvi_4.pkl', 'rb'))
print(pickled_model)

In [ ]:
# EB1: 0.9121361508267432
# EB2: 0.9152120215067565
# EB3: 0.9153888116216589
# EB4: 0.9334065797136774
# EB5: 0.9330483639924072
# EB6: 0.9325007563612594
# EB7: 13875 12950 0.9330562509477165
# EB8: 14220 13272 0.9119937102685194 
# EB9: 13875 12950 0.9318627771973583
# EB10: 14220 13272 0.9128319097402475

# %1: 359648 295422 0.9778067788170863
# %2: 359649 295421 0.977828848863204
# %3: 359649 295421 0.977814397217147
# %4: 359648 295421 0.9777926266177195
# %5: 359650 295422 0.9778923365411023
# %6: 359843 295233 0.9779668774041262
# %7: 359841 295230 0.977761799842581
# %8: 359840 296518 0.977231470922011
# %9: 359835 295223 0.977650365098458
# %10: 359837 295222 0.9778535497660246
